# Pytorch Tutorial

Pytorch is a python framework for machine learning

- GPU-accelerated computations
- automatic differentiation
- modules for neural networks

This tutorial will teach the fundamentals of operating on pytorch tensors and networks. You have already seen some things in recitation 0 which we will quickly review, but most of this tutorial is on mostly new or more advanced stuff.

For a worked example of how to build and train a pytorch network, see `pytorch-example.ipynb`.

For additional tutorials, see http://pytorch.org/tutorials/

In [1]:
import torch
import numpy as np
import torch.nn as nn

In [ ]:
torch.__version__

'1.10.2'

## Tensors

Tensors are the fundamental object for array data. The most common types you will use are `IntTensor` and `FloatTensor`.

In [ ]:
# Create uninitialized tensor
x = torch.FloatTensor(2,3)
print(x)
# Initialize to zeros
x.zero_()
print(x)

tensor([[7.5742e+31, 1.1576e+27, 4.5450e+30],
        [1.8524e+28, 8.7721e-43, 2.2457e-40]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [ ]:
# Create from numpy array (seed for repeatability)
np.random.seed(123)
np_array = np.random.random((2,3))
print(torch.FloatTensor(np_array))
print(torch.from_numpy(np_array))

tensor([[0.6965, 0.2861, 0.2269],
        [0.5513, 0.7195, 0.4231]])
tensor([[0.6965, 0.2861, 0.2269],
        [0.5513, 0.7195, 0.4231]], dtype=torch.float64)


In [ ]:
# Create random tensor (seed for repeatability)
torch.manual_seed(123)
x=torch.randn(2,3)
print(x)
# export to numpy array
x_np = x.numpy()
print(x_np)

tensor([[-0.1115,  0.1204, -0.3696],
        [-0.2404, -1.1969,  0.2093]])
[[-0.11146712  0.12036294 -0.3696345 ]
 [-0.24041797 -1.1969243   0.20926936]]


In [ ]:
# special tensors (see documentation)
print(torch.eye(3))
print(torch.ones(2,3))
print(torch.zeros(2,3))
print(torch.arange(0,3))

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0, 1, 2])


All tensors have a `size` and `type`

In [ ]:
x=torch.FloatTensor(3,4)
print(x.size())
print(x.type())

torch.Size([3, 4])
torch.FloatTensor


## Math, Linear Algebra, and Indexing (review)

Pytorch math and linear algebra is similar to numpy. Operators are overridden so you can use standard math operators (`+`,`-`, etc.) and expect a tensor as a result. See pytorch documentation for a complete list of available functions.

In [ ]:
x = torch.arange(0.,5.)
print(torch.sum(x))
print(torch.sum(torch.exp(x)))
print(torch.mean(x))

tensor(10.)
tensor(85.7910)
tensor(2.)


Pytorch indexing is similar to numpy indexing. See pytorch documentation for details.

In [ ]:
x = torch.rand(3,2)
print(x)
print(x[1,:])

tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])
tensor([0.3164, 0.4017])


## CPU and GPU

Tensors can be copied between CPU and GPU. It is important that everything involved in a calculation is on the same device.

This portion of the tutorial may not work for you if you do not have a GPU available.

In [ ]:
# create a tensor
x = torch.rand(3,2)
# copy to GPU
y = x.cuda()
# copy back to CPU
z = y.cpu()
# get CPU tensor as numpy array
# cannot get GPU tensor as numpy array directly
try:
    y.numpy()
except RuntimeError as e:
    print(e)

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

Operations between GPU and CPU tensors will fail. Operations require all arguments to be on the same device.

In [ ]:
x = torch.rand(3,5)  # CPU tensor
y = torch.rand(5,4).cuda()  # GPU tensor
try:
    torch.mm(x,y)  # Operation between CPU and GPU fails
except TypeError as e:
    print(e)

Typical code should include `if` statements or utilize helper functions so it can operate with or without the GPU.

In [ ]:
# Put tensor on CUDA if available
x = torch.rand(3,2)
if torch.cuda.is_available():
    x = x.cuda()
    print(x, x.dtype)

# Do some calculations
y = x ** 2
print(y)

# Copy to CPU if on GPU
if y.is_cuda:
    y = y.cpu()
    print(y, y.dtype)

tensor([[0.2745, 0.6584],
        [0.2775, 0.8573],
        [0.8993, 0.0390]], device='cuda:0') torch.float32
tensor([[0.0753, 0.4335],
        [0.0770, 0.7350],
        [0.8088, 0.0015]], device='cuda:0')
tensor([[0.0753, 0.4335],
        [0.0770, 0.7350],
        [0.8088, 0.0015]]) torch.float32


A convenient method is `new`, which creates a new tensor on the same device as another tensor. It should be used for creating tensors whenever possible.

In [ ]:
x1 = torch.rand(3,2)
x2 = x1.new(1,2)  # create cpu tensor
print(x2)
x1 = torch.rand(3,2).cuda()
x2 = x1.new(1,2)  # create cuda tensor
print(x2)

tensor([[0., 0.]])
tensor([[0.0753, 0.4335]], device='cuda:0')


Calculations executed on the GPU can be many times faster than numpy. However, numpy is still optimized for the CPU and many times faster than python `for` loops. Numpy calculations may be faster than GPU calculations for small arrays due to the cost of interfacing with the GPU.

In [ ]:
from timeit import timeit
# Create random data
x = torch.rand(1000,64)
y = torch.rand(64,32)
number = 10000  # number of iterations

def square():
    z=torch.mm(x, y) # dot product (mm=matrix multiplication)

# Time CPU
print('CPU: {}ms'.format(timeit(square, number=number)*1000))
# Time GPU
x, y = x.cuda(), y.cuda()
print('GPU: {}ms'.format(timeit(square, number=number)*1000))

## Differentiation

Tensors provide automatic differentiation.

As you might know, previous versions of Pytorch used Variables, which were wrappers around tensors for differentiation. Starting with pytorch 0.4.0, this wrapping is done internally in the Tensor class and you can, and should, differentiate Tensors directly. However, it is possible that you walk on references to Variables, e.g. in your error messages.

What you need to remember :

- Tensors you are differentiating with respect to must have `requires_grad=True`
- Call `.backward()` on scalar variables you are differentiating
- To differentiate a vector, sum it first

In [ ]:
# Create differentiable tensor
x = torch.tensor(torch.arange(0,4), requires_grad=False)
print(x.dtype)
# Calculate y=sum(x**2)
y = x**2
# Calculate gradient (dy/dx=2x)
y.sum().backward()
# Print values
print(x)
print(y)
print(x.grad)

C:\Users\Wendy\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.int64


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Differentiation accumulates gradients. This is sometimes what you want and sometimes not. **Make sure to zero gradients between batches if performing gradient descent or you will get strange results!**

In [ ]:
# Create a variable
x=torch.tensor(torch.arange(0,4), requires_grad=True)
# Differentiate
torch.sum(x**2).backward()
print(x.grad)
# Differentiate again (accumulates gradient)
torch.sum(x**2).backward()
print(x.grad)
# Zero gradient before differentiating
x.grad.data.zero_()
torch.sum(x**2).backward()
print(x.grad)

C:\Users\Wendy\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


RuntimeError: Only Tensors of floating point dtype can require gradients

Note that a Tensor with gradient cannot be exported to numpy directly :

In [ ]:
x=torch.tensor(torch.arange(0,4), requires_grad=True)
x.numpy() # raises an exception

C:\Users\Wendy\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


RuntimeError: Only Tensors of floating point dtype can require gradients

The reason is that pytorch remembers the graph of all computations to perform differenciation. To be integrated to this graph the raw data is wrapped internally to the Tensor class (like what was formerly a Variable). You can detach the tensor from the graph using the **.detach()** method, which returns a tensor with the same data but requires_grad set to False.

In [ ]:
x=torch.tensor(torch.arange(0,4), requires_grad=True)
y=x**2
z=y**2
z.detach().numpy()

C:\Users\Wendy\Anaconda3\envs\DL\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


RuntimeError: Only Tensors of floating point dtype can require gradients

Another reason to use this method is that updating the graph can use a lot of memory. If you are in a context where you have a differentiable tensor that you don't need to differentiate, think of detaching it from the graph.

## Neural Network Modules

Pytorch provides a framework for developing neural network modules. They take care of many things, the main one being wrapping and tracking a list of parameters for you.
You have several ways of building and using a network, offering different tradeoffs between freedom and simplicity.

torch.nn provides basic 1-layer nets, such as Linear (perceptron) and activation layers.

In [3]:
x = torch.arange(0,32).float()
net = torch.nn.Linear(32,10)
y = net(x)
print(y)

tensor([ 14.6551, -21.2337,  11.2941,   2.4101,  -8.7841,  -8.4820,  -4.6494,
         11.7071, -14.8966, -12.5824], grad_fn=<AddBackward0>)


All nn.Module objects are reusable as components of bigger networks ! That is how you build personnalized nets. The simplest way is to use the nn.Sequential class.

You can also create your own class that inherits n.Module. The forward method should precise what happens in the forward pass given an input. This enables you to precise behaviors more complicated than just applying layers one after another, if necessary.

In [4]:
# create a simple sequential network (`nn.Module` object) from layers (other `nn.Module` objects).
# Here a MLP with 2 layers and sigmoid activation.
net = torch.nn.Sequential(
    torch.nn.Linear(32,128),
    torch.nn.Sigmoid(),
    torch.nn.Linear(128,10))

In [6]:
# create a more customizable network module (equivalent here)
class MyNetwork(torch.nn.Module):
    # you can use the layer sizes as initialization arguments if you want to
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size,hidden_size)
        self.layer2 = torch.nn.Sigmoid()
        self.layer3 = torch.nn.Linear(hidden_size,output_size)

    def forward(self, input_val):
        h = input_val
        h = self.layer1(h)
        h = self.layer2(h)
        h = self.layer3(h)
        return h

net = MyNetwork(32,128,10)

The network tracks parameters, and you can access them through the **parameters()** method, which returns a python generator.

In [5]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0816, -0.0995, -0.1166,  ...,  0.1166,  0.0896,  0.0976],
        [ 0.1667,  0.0878,  0.1543,  ...,  0.0377, -0.0838, -0.1021],
        [ 0.0029,  0.1300,  0.0329,  ...,  0.0073,  0.0216, -0.1230],
        ...,
        [ 0.0614, -0.0652,  0.1360,  ...,  0.1361,  0.1089,  0.0750],
        [ 0.1105,  0.0236, -0.1050,  ...,  0.0810,  0.0951, -0.1092],
        [-0.1151, -0.0721,  0.0953,  ..., -0.0566,  0.1555,  0.0031]],
       requires_grad=True)
Parameter containing:
tensor([ 4.9386e-02, -5.2074e-02, -1.6802e-02,  4.3531e-02,  5.2061e-02,
        -4.6065e-02,  4.8371e-02, -8.5445e-02,  1.6534e-01,  5.4044e-02,
         1.3961e-01,  6.6039e-02,  3.2582e-02,  1.5769e-01,  7.1628e-02,
         1.2248e-01,  2.9231e-03, -1.1006e-01, -1.1526e-01,  3.9579e-02,
        -1.4732e-01, -8.8570e-02,  1.7245e-01, -1.2380e-01,  3.0472e-02,
        -1.1035e-01,  3.1870e-02, -1.3361e-01,  1.6717e-01,  2.2847e-02,
        -3.2548e-02, -8.8126e-02,  4.0940e-02,  2.0203e-0

Parameters are of type Parameter, which is basically a wrapper for a tensor. How does pytorch retrieve your network's parameters ? They are simply all the attributes of type Parameter in your network. Moreover, if an attribute is of type nn.Module, its own parameters are added to your network's parameters ! This is why, when you define a network by adding up basic components such as nn.Linear, you should never have to explicitely define parameters.

However, if you are in a case where no pytorch default module does what you need, you can define parameters explicitely (this should be rare). For the record, let's build the previous MLP with personnalized parameters.

In [ ]:
class MyNetworkWithParams(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(MyNetworkWithParams,self).__init__()
        self.layer1_weights = nn.Parameter(torch.randn(input_size,hidden_size))
        self.layer1_bias = nn.Parameter(torch.randn(hidden_size))
        self.layer2_weights = nn.Parameter(torch.randn(hidden_size,output_size))
        self.layer2_bias = nn.Parameter(torch.randn(output_size))

    def forward(self,x):
        h1 = torch.matmul(x,self.layer1_weights) + self.layer1_bias
        h1_act = torch.max(h1, torch.zeros(h1.size())) # ReLU
        output = torch.matmul(h1_act,self.layer2_weights) + self.layer2_bias
        return output

net = MyNetworkWithParams(32,128,10)

Parameters are useful in that they are meant to be all the network's weights that will be optimized during training. If you were needing to use a tensor in your computational graph that you want to remain constant, just define it as a regular tensor.

## Training

In [7]:
net = MyNetwork(32,128,10)

The nn.Module also provides loss functions, such as cross-entropy.

In [8]:
x = torch.tensor([np.arange(32), np.zeros(32),np.ones(32)]).float()
y = torch.tensor([0,3,9])
criterion = nn.CrossEntropyLoss()

output = net(x)
loss = criterion(output,y)
print(loss)

tensor(2.0366, grad_fn=<NllLossBackward0>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


nn.CrossEntropyLoss does both the softmax and the actual cross-entropy : given $output$ of size $(n,d)$ and $y$ of size $n$ and values in $0,1,...,d-1$, it computes $\sum_{i=0}^{n-1}log(s[i,y[i]])$ where $s[i,j] = \frac{e^{output[i,j]}}{\sum_{j'=0}^{d-1}e^{output[i,j']}}$

You can also compose nn.LogSoftmax and nn.NLLLoss to get the same result. Note that all these use the log-softmax rather than the softmax, for stability in the computations.

In [9]:
# equivalent
criterion2 = nn.NLLLoss()
sf = nn.LogSoftmax()
output = net(x)
loss = criterion(sf(output),y)
loss

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """


tensor(2.0366, grad_fn=<NllLossBackward0>)

Now, to perform the backward pass, just execute **loss.backward()** ! It will update gradients in all differentiable tensors in the graph, which in particular includes all the network parameters.

In [10]:
loss.backward()

# Check that the parameters now have gradients
for param in net.parameters():
    print(param.grad)

tensor([[-0.0033, -0.0024, -0.0015,  ...,  0.0234,  0.0243,  0.0252],
        [ 0.0083,  0.0083,  0.0083,  ...,  0.0083,  0.0083,  0.0083],
        [-0.0052, -0.0052, -0.0052,  ..., -0.0052, -0.0052, -0.0052],
        ...,
        [-0.0066, -0.0066, -0.0066,  ..., -0.0070, -0.0070, -0.0070],
        [-0.0008, -0.0030, -0.0052,  ..., -0.0659, -0.0682, -0.0704],
        [-0.0016, -0.0016, -0.0016,  ..., -0.0015, -0.0015, -0.0015]])
tensor([-5.9541e-03,  3.9260e-03, -1.0888e-02,  8.7817e-03, -1.3966e-03,
         1.0375e-02,  2.5833e-03,  7.1461e-03,  1.0315e-03,  8.5597e-04,
         3.5918e-03,  5.3241e-03,  4.0366e-03, -4.8465e-03, -7.4036e-03,
        -6.8651e-03,  1.1752e-02, -2.6479e-03,  3.3971e-03, -2.9414e-03,
         8.6274e-03, -5.6672e-03, -3.5430e-04, -2.8286e-03,  7.4300e-04,
         1.8761e-03, -3.4793e-03, -2.4353e-03, -3.7101e-03, -1.0119e-03,
        -4.3031e-03,  3.2497e-03,  2.3289e-04,  3.5754e-03, -3.7705e-03,
         6.6169e-03,  4.7236e-03, -5.9270e-03,  1.4396e

In [11]:
# if I forward prop and backward prop again, gradients accumulate :
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

# you can remove this behavior by reinitializing the gradients in your network parameters :
net.zero_grad()
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

tensor([[-0.0067, -0.0048, -0.0030,  ...,  0.0467,  0.0485,  0.0504],
        [ 0.0165,  0.0165,  0.0165,  ...,  0.0165,  0.0165,  0.0165],
        [-0.0104, -0.0104, -0.0104,  ..., -0.0104, -0.0104, -0.0104],
        ...,
        [-0.0132, -0.0132, -0.0133,  ..., -0.0140, -0.0140, -0.0141],
        [-0.0015, -0.0060, -0.0105,  ..., -0.1319, -0.1363, -0.1408],
        [-0.0031, -0.0031, -0.0031,  ..., -0.0031, -0.0031, -0.0031]])
tensor([-1.1908e-02,  7.8520e-03, -2.1776e-02,  1.7563e-02, -2.7932e-03,
         2.0751e-02,  5.1667e-03,  1.4292e-02,  2.0631e-03,  1.7119e-03,
         7.1836e-03,  1.0648e-02,  8.0732e-03, -9.6931e-03, -1.4807e-02,
        -1.3730e-02,  2.3504e-02, -5.2957e-03,  6.7943e-03, -5.8828e-03,
         1.7255e-02, -1.1334e-02, -7.0861e-04, -5.6573e-03,  1.4860e-03,
         3.7523e-03, -6.9587e-03, -4.8705e-03, -7.4203e-03, -2.0238e-03,
        -8.6062e-03,  6.4995e-03,  4.6577e-04,  7.1508e-03, -7.5410e-03,
         1.3234e-02,  9.4472e-03, -1.1854e-02,  2.8793e

We did backpropagation, but still didn't perform gradient descent. Let's define an optimizer on the network parameters.

In [12]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

print("Parameters before gradient descent :")
for param in net.parameters():
    print(param)

optimizer.step()

print("Parameters after gradient descent :")
for param in net.parameters():
    print(param)

Parameters before gradient descent :
Parameter containing:
tensor([[-0.1029, -0.1547, -0.1360,  ..., -0.0023, -0.1131,  0.0784],
        [ 0.1237, -0.1399,  0.0348,  ...,  0.0808, -0.0545, -0.1730],
        [ 0.1708,  0.0114, -0.0192,  ...,  0.1465, -0.1132, -0.1625],
        ...,
        [-0.0782,  0.1179,  0.0345,  ..., -0.1578, -0.0492, -0.1509],
        [-0.1569, -0.0867,  0.0415,  ..., -0.1076, -0.0669, -0.0068],
        [-0.0400,  0.0671,  0.1368,  ..., -0.0625,  0.1141,  0.1302]],
       requires_grad=True)
Parameter containing:
tensor([-0.0537, -0.0661,  0.0626,  0.1646,  0.0605, -0.1666,  0.0463,  0.0664,
        -0.0817,  0.1747, -0.0230, -0.0043,  0.1013, -0.1205, -0.0213, -0.0446,
         0.1154, -0.1111,  0.1758,  0.0344, -0.0786,  0.1729,  0.1386,  0.0203,
         0.1017, -0.0405, -0.1326,  0.1548, -0.0391, -0.1304,  0.0471, -0.0857,
         0.1218, -0.0449, -0.1258,  0.1711, -0.0497, -0.1714, -0.1058, -0.0260,
         0.1496, -0.1583, -0.0245, -0.0573, -0.0696, -0.05

In [ ]:
# In a training loop, we should perform many GD iterations.
n_iter = 1000
for i in range(n_iter):
    optimizer.zero_grad() # equivalent to net.zero_grad()
    output = net(x)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(2.0752, grad_fn=<NllLossBackward>)
tensor(1.9747, grad_fn=<NllLossBackward>)
tensor(1.8850, grad_fn=<NllLossBackward>)
tensor(1.8045, grad_fn=<NllLossBackward>)
tensor(1.7319, grad_fn=<NllLossBackward>)
tensor(1.6661, grad_fn=<NllLossBackward>)
tensor(1.6066, grad_fn=<NllLossBackward>)
tensor(1.5526, grad_fn=<NllLossBackward>)
tensor(1.5034, grad_fn=<NllLossBackward>)
tensor(1.4583, grad_fn=<NllLossBackward>)
tensor(1.4168, grad_fn=<NllLossBackward>)
tensor(1.3782, grad_fn=<NllLossBackward>)
tensor(1.3424, grad_fn=<NllLossBackward>)
tensor(1.3088, grad_fn=<NllLossBackward>)
tensor(1.2773, grad_fn=<NllLossBackward>)
tensor(1.2477, grad_fn=<NllLossBackward>)
tensor(1.2197, grad_fn=<NllLossBackward>)
tensor(1.1934, grad_fn=<NllLossBackward>)
tensor(1.1685, grad_fn=<NllLossBackward>)
tensor(1.1450, grad_fn=<NllLossBackward>)
tensor(1.1228, grad_fn=<NllLossBackward>)
tensor(1.1017, grad_fn=<NllLossBackward>)
tensor(1.0818, grad_fn=<NllLossBackward>)
tensor(1.0628, grad_fn=<NllLossBac

tensor(0.2130, grad_fn=<NllLossBackward>)
tensor(0.2125, grad_fn=<NllLossBackward>)
tensor(0.2119, grad_fn=<NllLossBackward>)
tensor(0.2114, grad_fn=<NllLossBackward>)
tensor(0.2108, grad_fn=<NllLossBackward>)
tensor(0.2103, grad_fn=<NllLossBackward>)
tensor(0.2097, grad_fn=<NllLossBackward>)
tensor(0.2092, grad_fn=<NllLossBackward>)
tensor(0.2086, grad_fn=<NllLossBackward>)
tensor(0.2081, grad_fn=<NllLossBackward>)
tensor(0.2075, grad_fn=<NllLossBackward>)
tensor(0.2070, grad_fn=<NllLossBackward>)
tensor(0.2065, grad_fn=<NllLossBackward>)
tensor(0.2059, grad_fn=<NllLossBackward>)
tensor(0.2054, grad_fn=<NllLossBackward>)
tensor(0.2049, grad_fn=<NllLossBackward>)
tensor(0.2044, grad_fn=<NllLossBackward>)
tensor(0.2038, grad_fn=<NllLossBackward>)
tensor(0.2033, grad_fn=<NllLossBackward>)
tensor(0.2028, grad_fn=<NllLossBackward>)
tensor(0.2023, grad_fn=<NllLossBackward>)
tensor(0.2017, grad_fn=<NllLossBackward>)
tensor(0.2012, grad_fn=<NllLossBackward>)
tensor(0.2007, grad_fn=<NllLossBac

tensor(0.0921, grad_fn=<NllLossBackward>)
tensor(0.0920, grad_fn=<NllLossBackward>)
tensor(0.0918, grad_fn=<NllLossBackward>)
tensor(0.0916, grad_fn=<NllLossBackward>)
tensor(0.0915, grad_fn=<NllLossBackward>)
tensor(0.0913, grad_fn=<NllLossBackward>)
tensor(0.0912, grad_fn=<NllLossBackward>)
tensor(0.0910, grad_fn=<NllLossBackward>)
tensor(0.0909, grad_fn=<NllLossBackward>)
tensor(0.0907, grad_fn=<NllLossBackward>)
tensor(0.0906, grad_fn=<NllLossBackward>)
tensor(0.0904, grad_fn=<NllLossBackward>)
tensor(0.0902, grad_fn=<NllLossBackward>)
tensor(0.0901, grad_fn=<NllLossBackward>)
tensor(0.0899, grad_fn=<NllLossBackward>)
tensor(0.0898, grad_fn=<NllLossBackward>)
tensor(0.0896, grad_fn=<NllLossBackward>)
tensor(0.0895, grad_fn=<NllLossBackward>)
tensor(0.0893, grad_fn=<NllLossBackward>)
tensor(0.0892, grad_fn=<NllLossBackward>)
tensor(0.0890, grad_fn=<NllLossBackward>)
tensor(0.0889, grad_fn=<NllLossBackward>)
tensor(0.0887, grad_fn=<NllLossBackward>)
tensor(0.0886, grad_fn=<NllLossBac

In [ ]:
output = net(x)
print(output)
print(y)

tensor([[ 7.9902, -1.7800, -2.0774,  0.5787, -1.9603, -1.7135, -1.6187, -1.8039,
         -1.8273,  4.1082],
        [ 0.2567, -1.4705, -1.5120,  5.7903, -1.5307, -1.6327, -1.5679, -1.5251,
         -1.6856,  3.2233],
        [ 1.8633, -1.4923, -1.5976,  3.0309, -1.6929, -1.5278, -1.5327, -1.6354,
         -1.6566,  5.6728]], grad_fn=<AddmmBackward>)
tensor([0, 3, 9])


Now you know how to train a network ! For a complete training check the pytorch_example notebook.

all in one block:

In [18]:
from torch import optim
x=torch.tensor([np.arange(32),np.zeros(32),np.ones(32)]).float()
y=torch.tensor([0,3,9])
net=nn.Sequential(nn.Linear(32,128),nn.ReLU(),nn.Linear(128,10))
critierion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters(),lr=0.1)
for i in range(100):
    optimizer.zero_grad()
    output=net(x)
    loss=criterion(output,y)
    loss.backward()
    print(loss)
    optimizer.step()


tensor(4.1786, grad_fn=<NllLossBackward0>)
tensor(7.0213, grad_fn=<NllLossBackward0>)
tensor(2.4124, grad_fn=<NllLossBackward0>)
tensor(1.2926, grad_fn=<NllLossBackward0>)
tensor(1.0431, grad_fn=<NllLossBackward0>)
tensor(0.8928, grad_fn=<NllLossBackward0>)
tensor(0.8029, grad_fn=<NllLossBackward0>)
tensor(0.7822, grad_fn=<NllLossBackward0>)
tensor(0.7105, grad_fn=<NllLossBackward0>)
tensor(0.6862, grad_fn=<NllLossBackward0>)
tensor(0.9645, grad_fn=<NllLossBackward0>)
tensor(0.6285, grad_fn=<NllLossBackward0>)
tensor(0.6016, grad_fn=<NllLossBackward0>)
tensor(0.5786, grad_fn=<NllLossBackward0>)
tensor(0.5574, grad_fn=<NllLossBackward0>)
tensor(0.5375, grad_fn=<NllLossBackward0>)
tensor(0.5186, grad_fn=<NllLossBackward0>)
tensor(0.5007, grad_fn=<NllLossBackward0>)
tensor(0.4835, grad_fn=<NllLossBackward0>)
tensor(0.4666, grad_fn=<NllLossBackward0>)
tensor(0.4501, grad_fn=<NllLossBackward0>)
tensor(0.4341, grad_fn=<NllLossBackward0>)
tensor(0.4184, grad_fn=<NllLossBackward0>)
tensor(0.40

## Saving and Loading

In [ ]:
# get dictionary of keys to weights using `state_dict`
net = torch.nn.Sequential(
    torch.nn.Linear(28*28,256),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256,10))
print(net.state_dict().keys())

odict_keys(['0.weight', '0.bias', '2.weight', '2.bias'])


In [ ]:
# save a dictionary
torch.save(net.state_dict(),'test.t7')
# load a dictionary
net.load_state_dict(torch.load('test.t7'))

<All keys matched successfully>

## Common issues to look out for

### Type mismatch

In [ ]:
net = nn.Linear(4,2)
x = torch.tensor([1,2,3,4])
y = net(x)
print(y)

RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2'

In [ ]:
x = x.float()
x = torch.tensor([1.,2.,3.,4.])

In [ ]:
x = 2* torch.ones(2,2)
y = 3* torch.ones(2,2)
print(x * y)
print(x.matmul(y))

tensor([[6., 6.],
        [6., 6.]])
tensor([[12., 12.],
        [12., 12.]])


In [ ]:
x = torch.ones(4,5)
y = torch.arange(5)
print(x+y)
y = torch.arange(4).view(-1,1)
print(x+y)
y = torch.arange(4)
print(x+y) # exception

RuntimeError: expected device cpu and dtype Float but got device cpu and dtype Long

In [ ]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)
print(x.t())
print(x.view(3,2))

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [ ]:
net = nn.Sequential(nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,120))
x = torch.ones(256,2048)
y = torch.zeros(256).long()
net.cuda()
x.cuda()
crit=nn.CrossEntropyLoss()
out = net(x)
loss = crit(out,y)
loss.backward()

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #4 'mat1'

In [ ]:
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))


    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out

In [ ]:
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
        self.hidden = nn.ModuleList(self.hidden)

    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out